# OpenLCA Database Structure

This document describes the structure of an openLCA Derby database. In particular, we discuss how openLCA handles location, unit and cost data. We describe how to access this information using the low-level functions in the `mola` package.

The `mola` package converts the Derby database to SQLite to make it easier to access in Python. The structure of Derby and SQLite databases are the same, so we refer to the SQLite database here.

First, we make a connection to the default Ecoinvent SQLITE database using `mola` and show the tables in the database.

In [1]:
import mola.dataimport as di
import mola.dataview as dv

conn = di.get_sqlite_connection()
dv.get_table_names(conn)

SELECT "name" FROM "sqlite_master" WHERE "type"='table' AND "name" NOT LIKE 'sqlite_'


['TBL_ACTORS',
 'TBL_CATEGORIES',
 'TBL_CURRENCIES',
 'TBL_DQ_INDICATORS',
 'TBL_DQ_SCORES',
 'TBL_DQ_SYSTEMS',
 'TBL_EXCHANGES',
 'TBL_FLOWS',
 'TBL_FLOW_PROPERTIES',
 'TBL_FLOW_PROPERTY_FACTORS',
 'TBL_IMPACT_CATEGORIES',
 'TBL_IMPACT_FACTORS',
 'TBL_IMPACT_METHODS',
 'TBL_LOCATIONS',
 'TBL_PROCESSES',
 'TBL_PROCESS_DOCS',
 'TBL_PROCESS_LINKS',
 'TBL_PRODUCT_SYSTEMS',
 'TBL_PRODUCT_SYSTEM_PROCESSES',
 'TBL_SOURCES',
 'TBL_UNITS',
 'TBL_UNIT_GROUPS']

# Processes

A reference output flow (or product flow) for an openLCA process is called its quantitative reference. OpenLCA multioutput processes have multiple reference flows. In the database, the processes are split so that they only have a single reference flow (quantitative reference). For example, off-shore petroleum and gas production in GB has two processes:

Reference ID | Process | Product Flow | Country
-- | -- | -- | --
0a89372d-eb22-325b-a740-8127055406a1 | petroleum and gas production, off-shore | natural gas, high pressure | APOS, S - GB
927fcc91-4d4c-36a8-a6d2-7e8f0c8d6f40 | petroleum and gas production, off-shore | petroleum | APOS, S - GB

In [2]:
ref_id = ['0a89372d-eb22-325b-a740-8127055406a1', '927fcc91-4d4c-36a8-a6d2-7e8f0c8d6f40']
dv.get_process_product_flow(conn, ref_id)

SELECT "TBL_PROCESSES"."REF_ID" "PROCESS_REF_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","TBL_LOCATIONS"."NAME" "LOCATION","TBL_FLOWS"."REF_ID" "FLOW_REF_ID","TBL_FLOWS"."NAME" "FLOW_NAME" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (SELECT "ID" FROM "TBL_PROCESSES" WHERE "REF_ID" IN ('0a89372d-eb22-325b-a740-8127055406a1','927fcc91-4d4c-36a8-a6d2-7e8f0c8d6f40'))) "sq0" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="sq0"."F_FLOW" LEFT JOIN "TBL_PROCESSES" ON "TBL_PROCESSES"."ID"="sq0"."F_OWNER" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_FLOWS"."FlOW_TYPE"='PRODUCT_FLOW'


,PROCESS_REF_ID,PROCESS_NAME,LOCATION,FLOW_REF_ID,FLOW_NAME
0,0a89372d-eb22-325b-a740-8127055406a1,"petroleum and gas production, off-shore | natu...",United Kingdom,a9007f10-7e39-4d50-8f4a-d6d03ce3d673,"natural gas, high pressure"
1,927fcc91-4d4c-36a8-a6d2-7e8f0c8d6f40,"petroleum and gas production, off-shore | petr...",United Kingdom,e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,petroleum


# Location

Location data in openLCA is held in the processes table and not in the flows table. The previous section shows how we can retrieve location from the processes using `mola`. 

We show the top 5 rows of the flows table below.

In [3]:
flows_dfr = dv.get_table(conn, 'TBL_FLOWS')
flows_dfr.head()

SELECT * FROM "TBL_FLOWS"


,ID,REF_ID,NAME,VERSION,LAST_CHANGE,F_CATEGORY,SYNONYMS,DESCRIPTION,FLOW_TYPE,INFRASTRUCTURE_FLOW,CAS_NUMBER,FORMULA,F_REFERENCE_FLOW_PROPERTY,F_LOCATION
0,125152,dcb70fd5-e305-4920-b68b-40697cc6a206,waste polyethylene terephthalate,3,1575311732534,125154.0,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,WASTE_FLOW,0,None,None,1251,None
1,125157,0f440cc0-0f74-446d-99d6-8ff0e97a2444,Ammonia,5,1602495209690,125156.0,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,ELEMENTARY_FLOW,0,None,None,1251,None
2,125159,e336eee7-148a-4d1c-8027-780cbfafa12b,Copper,5,1602495209697,125156.0,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,ELEMENTARY_FLOW,0,None,None,1251,None
3,125161,e6551223-73b6-4289-b841-c5cdeb25abd9,Selenium,5,1602495209702,125156.0,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,ELEMENTARY_FLOW,0,None,None,1251,None
4,125163,33b38ccb-593b-4b11-b965-10d747ba3556,"NMVOC, non-methane volatile organic compounds,...",5,1602495209708,125156.0,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,ELEMENTARY_FLOW,0,None,None,1251,None


The `TBL_FLOWS` table has a `F_LOCATION` foreign key, but for the `lemon` flow it not populated. One must use the process table and find the process that generated the flow.

In [4]:
flows_dfr[flows_dfr.NAME=='lemon']

,ID,REF_ID,NAME,VERSION,LAST_CHANGE,F_CATEGORY,SYNONYMS,DESCRIPTION,FLOW_TYPE,INFRASTRUCTURE_FLOW,CAS_NUMBER,FORMULA,F_REFERENCE_FLOW_PROPERTY,F_LOCATION
3235,132016,9891d347-28f8-434d-896a-84a19a4c68a6,lemon,3,1575311718846,129720.0,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,PRODUCT_FLOW,0,None,None,1251,None


In fact, it is not populated for any flow suggesting that it is redundant (at least for the Ecoinvent database).

In [5]:
flows_dfr[~flows_dfr.F_LOCATION.isnull()]

,ID,REF_ID,NAME,VERSION,LAST_CHANGE,F_CATEGORY,SYNONYMS,DESCRIPTION,FLOW_TYPE,INFRASTRUCTURE_FLOW,CAS_NUMBER,FORMULA,F_REFERENCE_FLOW_PROPERTY,F_LOCATION


# Units

Unit data is held in the tables `TBL_UNITS` and `TBL_UNIT_GROUPS`. We load the tables into this notebook.

In [6]:
units_dfr = dv.get_table(conn, 'TBL_UNITS')
units_dfr

SELECT * FROM "TBL_UNITS"


,ID,REF_ID,NAME,DESCRIPTION,VERSION,LAST_CHANGE,CONVERSION_FACTOR,SYNONYMS,F_UNIT_GROUP
0,1016,007f0ce1-4a0b-4bb2-8f7c-1b2a0e6ef6b2,LVL 2000,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,NaN,NaN,1.780000e+00,None,1017
1,1018,00d8370e-2bf1-4f3b-81bb-f8f147e84819,m2*d,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,NaN,NaN,2.740000e-03,None,1019
2,1020,010f811e-3cc2-4b14-a901-337da9b3e49c,kcal,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,NaN,NaN,4.190000e-03,None,1021
3,1022,01e58eb9-0aba-4c76-ba0c-03f6f3be1353,GJ,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,NaN,NaN,1.000000e+03,None,1021
4,1023,0300ec69-ce1a-45f0-bcf0-7b33845dc53e,lb av,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,NaN,NaN,4.500000e-01,lb,1024
...,...,...,...,...,...,...,...,...,...
186,1229,fd100082-4cdf-4932-9282-6bb7d7091bd4,CZK 2000,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,NaN,NaN,2.000000e-02,None,1017
187,1230,fe8da65d-f0ea-4496-b13e-1955aaa412d7,p*km,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,NaN,NaN,1.000000e+00,pkm;personkm,1117
188,1231,fea33583-ae83-4398-b7fb-7f90fd097269,qt (US liq),C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,NaN,NaN,9.463530e-04,None,1028
189,1232,ff1a332f-967c-4084-8b01-8d3ca81b3121,PJ,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,NaN,NaN,1.000000e+09,None,1021


In [7]:
unit_groups = dv.get_table(conn, 'TBL_UNIT_GROUPS')
unit_groups

SELECT * FROM "TBL_UNIT_GROUPS"


,ID,REF_ID,NAME,VERSION,LAST_CHANGE,F_CATEGORY,DESCRIPTION,F_REFERENCE_UNIT,F_DEFAULT_FLOW_PROPERTY
0,1117,11d161f0-37e3-4d49-bf7a-ff4f31a9e5c7,Units of person transport,NaN,NaN,1011.0,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1230,1233
1,1150,258d6abd-14f2-4484-956c-c88e8f6fd8ed,Units of energy/mass*time,NaN,NaN,1011.0,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1149,1234
2,1125,326eb58b-e5b3-4cea-b45a-2398c25109f8,Units of length*time,NaN,NaN,1011.0,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1224,1235
3,1041,36932b14-ba61-417b-a80c-eb9935d193f1,Units of length*area,NaN,NaN,1011.0,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1040,1236
4,1111,3dbb60e1-edde-49f7-b28d-f34b4af727b3,Units of mole*area/mass,NaN,NaN,1011.0,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1110,1237
5,1067,5454b231-270e-45e6-89b2-7f4f3e482245,Units of items*length,NaN,NaN,1011.0,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1066,1238
6,1061,59f191d6-5dd3-4553-af88-1a32accfe308,Units of mass*time,NaN,NaN,1011.0,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1182,1239
7,1076,59f6a0a2-731f-41c3-86df-d383dc673dfe,Units of mechanical filtration (occ.),NaN,NaN,1011.0,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1075,1240
8,1082,5beb6eed-33a9-47b8-9ede-1dfe8f679159,Units of items,NaN,NaN,1011.0,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1115,1241
9,1033,838aaa21-0117-11db-92e3-0800200c9a66,Units of mass*length,NaN,NaN,1011.0,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,1042,1242


## Impact category

We retrieve the reference id for an impact category and obtain the reference unit for its coefficients.

In [8]:
impact_category_dfr = dv.get_impact_categories(conn, method_name=['ReCiPe Midpoint (H) V1.13'], category_name=['fossil depletion - FDP'])
impact_category_dfr

SELECT "TBL_IMPACT_METHODS"."ID" "methods_ID","TBL_IMPACT_METHODS"."REF_ID" "methods_REF_ID","TBL_IMPACT_METHODS"."NAME" "methods_NAME","TBL_IMPACT_CATEGORIES"."ID" "categories_ID","TBL_IMPACT_CATEGORIES"."REF_ID" "categories_REF_ID","TBL_IMPACT_CATEGORIES"."NAME" "categories_NAME","TBL_IMPACT_CATEGORIES"."REFERENCE_UNIT" "categories_REFERENCE_UNIT" FROM "TBL_IMPACT_CATEGORIES" LEFT JOIN "TBL_IMPACT_METHODS" ON "TBL_IMPACT_CATEGORIES"."F_IMPACT_METHOD"="TBL_IMPACT_METHODS"."ID" WHERE "TBL_IMPACT_METHODS"."name" LIKE 'ReCiPe Midpoint (H) V1.13' AND "TBL_IMPACT_CATEGORIES"."name" LIKE 'fossil depletion - FDP'


,methods_ID,methods_REF_ID,methods_NAME,categories_ID,categories_REF_ID,categories_NAME,categories_REFERENCE_UNIT
0,36132594,dc7995f0-623b-41a2-ad70-bf62a1b5e652,ReCiPe Midpoint (H) V1.13,36139540,3bb7b9c4-6934-325c-974c-12d2ed74407b,fossil depletion - FDP,kg oil-Eq


The `REFERENCE_UNIT` here is just a text string and gives the unit of impact. There are a set of coefficients for each impact category. We obtain the units for each of these coefficients below. They give the impact per unit of elementary flow.

In [9]:
impact_coeffs_dfr = dv.get_impact_category_elementary_flow(conn, ref_ids=['3bb7b9c4-6934-325c-974c-12d2ed74407b'])
impact_coeffs_units_dfr = impact_coeffs_dfr.stack().reset_index()
impact_coeffs_units_dfr

SELECT "ic"."REF_ID" "IMPACT_CATEGORY_REF_ID","TBL_FLOWS"."REF_ID" "FLOW_REF_ID","TBL_IMPACT_FACTORS"."VALUE","TBL_IMPACT_FACTORS"."F_UNIT" FROM (SELECT "ID","REF_ID" FROM "TBL_IMPACT_CATEGORIES" WHERE "REF_ID" IN ('3bb7b9c4-6934-325c-974c-12d2ed74407b')) "ic" LEFT JOIN "TBL_IMPACT_FACTORS" ON "TBL_IMPACT_FACTORS"."F_IMPACT_CATEGORY"="ic"."ID" LEFT JOIN "TBL_FLOWS" ON "TBL_IMPACT_FACTORS"."F_FLOW"="TBL_FLOWS"."ID"


IMPACT_CATEGORY_REF_ID,FLOW_REF_ID,F_UNIT,3bb7b9c4-6934-325c-974c-12d2ed74407b
0,024c9722-1e88-412b-8c4b-10c532be8dca,1056,0.225
1,3ed5f377-344f-423a-b5ec-9a9a1162b944,1052,1.070
2,7c337428-fb1b-45c7-bbb2-2ee4d29e17ba,1052,1.110
3,88d06db9-59a1-4719-9174-afeb1fa4026a,1056,1.040
4,b6d0042d-0ef8-49ed-9162-a07ff1ccf750,1056,0.434


If we join the elementary flow and unit tables we can see the names of the flows and units.

In [10]:
icu_dfr = impact_coeffs_units_dfr.merge(units_dfr[['ID', 'NAME']], left_on='F_UNIT', right_on='ID')
icu_dfr.columns = ['REF_ID', 'F_UNIT', 'FOSSIL_DEPLETION', 'ID', 'UNIT_NAME']
elementary_flows_dfr = dv.get_elementary_flows(conn)
icu_dfr = icu_dfr.merge(elementary_flows_dfr[['REF_ID', 'NAME']], on='REF_ID')
icu_dfr = icu_dfr[['REF_ID', 'NAME', 'FOSSIL_DEPLETION', 'UNIT_NAME']].set_index('REF_ID')
icu_dfr

SELECT "ID","REF_ID","NAME" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='ELEMENTARY_FLOW'


,NAME,FOSSIL_DEPLETION,UNIT_NAME
REF_ID,,,
024c9722-1e88-412b-8c4b-10c532be8dca,"Coal, brown, in ground",0.225,kg
88d06db9-59a1-4719-9174-afeb1fa4026a,"Oil, crude, in ground",1.040,kg
b6d0042d-0ef8-49ed-9162-a07ff1ccf750,"Coal, hard, unspecified, in ground",0.434,kg
3ed5f377-344f-423a-b5ec-9a9a1162b944,"Gas, mine, off-gas, process, coal mining",1.070,m3
7c337428-fb1b-45c7-bbb2-2ee4d29e17ba,"Gas, natural, in ground",1.110,m3


## Processes and flows

Now we find a relevant process and its breakdown into elementary flows.

In [11]:
oil_production_dfr = dv.get_processes(conn, name=['petroleum production, onshore%'])
oil_production_dfr

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE 'petroleum production, onshore%'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,7076520,81d22f90-89fc-3f04-845e-97615867d8f9,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Russian Federation,7076522
1,10012499,2c08823a-4575-30ba-bf61-36e2fd4d56e6,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Rest-of-World,10012501
2,15742513,c1827905-bfad-3fa1-b00b-1303f970ae92,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Middle East,15742515
3,29432025,cd177b7d-e908-3e69-b40c-4827b4abaa4d,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Africa,29432027


In [12]:
middle_east_breakdown_dfr = dv.get_process_elementary_flow(conn, ref_ids='c1827905-bfad-3fa1-b00b-1303f970ae92')
middle_east_breakdown_dfr

SELECT "e"."F_OWNER","e"."F_FLOW","e"."F_UNIT","e"."RESULTING_AMOUNT_VALUE","TBL_FLOWS"."FLOW_TYPE","TBL_FLOWS"."REF_ID" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (15742513)) "e" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="e"."F_FLOW" WHERE "TBL_FLOWS"."FlOW_TYPE"='ELEMENTARY_FLOW'


F_OWNER                              c1827905-bfad-3fa1-b00b-1303f970ae92  \
F_UNIT                                                               1056   
REF_ID                                                                      
00012c0a-9bff-4787-a7eb-56c3d2f43692                         3.190751e-14   
0015ec22-72cb-4af1-8c7b-0ba0d041553c                         2.310698e-14   
0017271e-7df5-40bc-833a-36110c1fe5d5                         1.335248e-08   
001790f3-fd86-4a0d-a2a1-06c7099d90c8                                  NaN   
0017ce28-9f7a-404b-ad55-d3f43ad13cae                         4.346948e-16   
...                                                                   ...   
ff36578b-f403-4656-b934-81d8d4e02dc8                         2.137035e-09   
ff65658c-2d39-44a2-b23e-7ec3c644f064                         1.781273e-06   
ff6dccc1-5ebd-42c3-9fd9-3d73db7a3dd2                                  NaN   
ffaaffd3-5deb-4508-9e5f-e47f551ac2b8                         6.656193e-15   
ffbe8447-6d78-46dd-8378-47019a82ce82                                  NaN   

F_OWNER                                                                     \
F_UNIT                                        1217 1052 1191          1084   
REF_ID                                                                       
00012c0a-9bff-4787-a7eb-56c3d2f43692           NaN  NaN  NaN           NaN   
0015ec22-72cb-4af1-8c7b-0ba0d041553c           NaN  NaN  NaN           NaN   
0017271e-7df5-40bc-833a-36110c1fe5d5           NaN  NaN  NaN           NaN   
001790f3-fd86-4a0d-a2a1-06c7099d90c8  1.173071e-05  NaN  NaN           NaN   
0017ce28-9f7a-404b-ad55-d3f43ad13cae           NaN  NaN  NaN           NaN   
...                                            ...  ...  ...           ...   
ff36578b-f403-4656-b934-81d8d4e02dc8           NaN  NaN  NaN           NaN   
ff65658c-2d39-44a2-b23e-7ec3c644f064           NaN  NaN  NaN           NaN   
ff6dccc1-5ebd-42c3-9fd9-3d73db7a3dd2           NaN  NaN  NaN  1.082345e-11   
ffaaffd3-5deb-4508-9e5f-e47f551ac2b8           NaN  NaN  NaN           NaN   
ffbe8447-6d78-46dd-8378-47019a82ce82  4.712760e-11  NaN  NaN           NaN   

F_OWNER                                         
F_UNIT                               1098 1218  
REF_ID                                          
00012c0a-9bff-4787-a7eb-56c3d2f43692  NaN  NaN  
0015ec22-72cb-4af1-8c7b-0ba0d041553c  NaN  NaN  
0017271e-7df5-40bc-833a-36110c1fe5d5  NaN  NaN  
001790f3-fd86-4a0d-a2a1-06c7099d90c8  NaN  NaN  
0017ce28-9f7a-404b-ad55-d3f43ad13cae  NaN  NaN  
...                                   ...  ...  
ff36578b-f403-4656-b934-81d8d4e02dc8  NaN  NaN  
ff65658c-2d39-44a2-b23e-7ec3c644f064  NaN  NaN  
ff6dccc1-5ebd-42c3-9fd9-3d73db7a3dd2  NaN  NaN  
ffaaffd3-5deb-4508-9e5f-e47f551ac2b8  NaN  NaN  
ffbe8447-6d78-46dd-8378-47019a82ce82  NaN  NaN  

[2046 rows x 7 columns]

In [13]:
meb_dfr = middle_east_breakdown_dfr.stack().reset_index()
meb_dfr.columns = ['REF_ID', 'F_UNIT', 'MIDDLE_EAST_OIL']
meb_dfr = meb_dfr.merge(elementary_flows_dfr[['REF_ID', 'NAME']], on='REF_ID')
meb_dfr = meb_dfr.merge(units_dfr[['ID', 'NAME']], left_on='F_UNIT', right_on='ID')
meb_dfr = meb_dfr[['REF_ID', 'MIDDLE_EAST_OIL', 'NAME_x', 'NAME_y']]
meb_dfr.columns = ['REF_ID', 'MIDDLE_EAST_OIL', 'FLOW_NAME', 'UNIT_NAME']
meb_dfr = meb_dfr[['REF_ID', 'FLOW_NAME', 'MIDDLE_EAST_OIL', 'UNIT_NAME']].set_index('REF_ID')
meb_dfr

,FLOW_NAME,MIDDLE_EAST_OIL,UNIT_NAME
REF_ID,,,
00012c0a-9bff-4787-a7eb-56c3d2f43692,Metolachlor,3.190751e-14,kg
0015ec22-72cb-4af1-8c7b-0ba0d041553c,Florasulam,2.310698e-14,kg
0017271e-7df5-40bc-833a-36110c1fe5d5,Nitrite,1.335248e-08,kg
0017ce28-9f7a-404b-ad55-d3f43ad13cae,Benzo(ghi)perylene,4.346948e-16,kg
006aa3f7-59ba-450f-aa45-a2b2d1752647,Sulfate,1.264261e-09,kg
...,...,...,...
f14b59ff-d438-442d-8bad-b53694b8263a,Water,4.801550e-15,m3
f6df5030-8e06-4276-bfeb-219db8dab104,"Volume occupied, final repository for low-acti...",1.997059e-07,m3
f977a02e-3564-4798-843c-9fb9a18bc18b,Water,1.594224e-07,m3


We merge the process elementary breakdown with the impact coefficient table. Since there are only five coefficients for 'FOSSIL DEPLETION' there are only five relevant elementary flows.

In [14]:
merge_dfr = meb_dfr.merge(icu_dfr, on='REF_ID')
merge_dfr

,FLOW_NAME,MIDDLE_EAST_OIL,UNIT_NAME_x,NAME,FOSSIL_DEPLETION,UNIT_NAME_y
REF_ID,,,,,,
024c9722-1e88-412b-8c4b-10c532be8dca,"Coal, brown, in ground",0.001303,kg,"Coal, brown, in ground",0.225,kg
88d06db9-59a1-4719-9174-afeb1fa4026a,"Oil, crude, in ground",1.011293,kg,"Oil, crude, in ground",1.040,kg
b6d0042d-0ef8-49ed-9162-a07ff1ccf750,"Coal, hard, unspecified, in ground",0.013188,kg,"Coal, hard, unspecified, in ground",0.434,kg
3ed5f377-344f-423a-b5ec-9a9a1162b944,"Gas, mine, off-gas, process, coal mining",0.000113,m3,"Gas, mine, off-gas, process, coal mining",1.070,m3
7c337428-fb1b-45c7-bbb2-2ee4d29e17ba,"Gas, natural, in ground",0.016648,m3,"Gas, natural, in ground",1.110,m3


We can see the units match up so the impact of middle east oil production on fossil depletion in the reference unit `kg oil-Eq` is

In [15]:
sum(merge_dfr['MIDDLE_EAST_OIL'] * merge_dfr['FOSSIL_DEPLETION'])

1.0763621196391

We can see that this result agrees with the openLCA client application as shown below.

![Fossil_Depletion](Figures/Fossil_Depletion.png)

## Pyomo

In pyomo, units are implemented using the `pint` Python package. 

In [16]:
from pint import UnitRegistry
ureg = UnitRegistry()
distance = 24.0*ureg.meter
distance

24.0 <Unit('meter')>

In [17]:
10*ureg.kiloBq

10 <Unit('kilobecquerel')>

In [18]:
10* ureg.count

10 <Unit('count')>

The `mola` package implements units in the abstract model of the Specification class so the reference units are defined by the model.

The package contains a `Lemon_Toy_Model` configuration file from which we can build a concrete Pyomo model.

In [19]:
import mola.specification5 as ms
import mola.build as mb
from pyomo.environ import units as pu
spec = ms.ScheduleSpecification()
config = mb.get_config('../config/Lemon_Toy_Model.json')
model = mb.build_instance(config)

The concrete model contains a distance parameter $d$.

In [20]:
model.d.pprint()

d : Distance
    Size=1, Index=d_index, Domain=Reals, Default=None, Mutable=True
    Key                                                                                          : Value
    ('64867712-23c4-3be5-a50e-3631e74571a6', '9891d347-28f8-434d-896a-84a19a4c68a6', 'k1', 't1') :     0


We can see that the distance parameter $d$ has reference units of `km`.

In [21]:
d_units = pu.get_units(model.d['64867712-23c4-3be5-a50e-3631e74571a6', '9891d347-28f8-434d-896a-84a19a4c68a6', 'k1', 't1'])
d_units.pprint()

km

After the build phase we can obtain the `Flow` variable, but at the moment it does not explicitly contains units because they are not loaded from the database.

In [22]:
model.Flow.pprint()

Flow : Material flow
    Size=1, Index=Flow_index
    Key                                                                                          : Lower : Value : Upper : Fixed : Stale : Domain
    ('9891d347-28f8-434d-896a-84a19a4c68a6', '64867712-23c4-3be5-a50e-3631e74571a6', 'k1', 't1') :     0 :  None :  None : False :  True : NonNegativeReals


In [23]:
pu.get_units(model.Flow).pprint()

F_m

However, we can use a `mola` function to find the units based on the process reference id.

In [24]:
dv.get_process_product_flow_units(conn, ['64867712-23c4-3be5-a50e-3631e74571a6'])

SELECT "TBL_PROCESSES"."REF_ID" "P","TBL_UNITS"."NAME" "UNITS" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (SELECT "ID" FROM "TBL_PROCESSES" WHERE "REF_ID" IN ('64867712-23c4-3be5-a50e-3631e74571a6'))) "sq0" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="sq0"."F_FLOW" LEFT JOIN "TBL_PROCESSES" ON "TBL_PROCESSES"."ID"="sq0"."F_OWNER" LEFT JOIN "TBL_UNITS" ON "TBL_UNITS"."ID"="sq0"."F_UNIT" WHERE "TBL_FLOWS"."FlOW_TYPE"='PRODUCT_FLOW'


,UNITS
P,
64867712-23c4-3be5-a50e-3631e74571a6,kg


 The output material flow is in `kg`.
 
 Let's run the optimiser using `pyomo`.

In [25]:
import pyomo.environ as pe
opt = pe.SolverFactory("glpk")
model.obj1.activate()
model.obj2.deactivate()
model.obj.deactivate()
opt.solve(model)
model.Flow.pprint()

Flow : Material flow
    Size=1, Index=Flow_index
    Key                                                                                          : Lower : Value  : Upper : Fixed : Stale : Domain
    ('9891d347-28f8-434d-896a-84a19a4c68a6', '64867712-23c4-3be5-a50e-3631e74571a6', 'k1', 't1') :     0 : 1000.0 :  None : False : False : NonNegativeReals


 We can use a `mola` output function to incorporate `Flow` units.

In [26]:
import mola.output as mo
lookup = dv.LookupTables(conn)
mo.get_entity(model.Flow, lookup=lookup, drop=True, units=['P_m'])

SELECT "REF_ID","NAME" FROM "TBL_CATEGORIES"
SELECT "REF_ID" "FLOW_REF_ID","NAME" FROM "TBL_FLOWS"
SELECT "TBL_PROCESSES"."REF_ID" "PROCESS_REF_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","TBL_LOCATIONS"."NAME" "LOCATION_NAME" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID"
SELECT "REF_ID","NAME" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='PRODUCT_FLOW'
SELECT "TBL_IMPACT_METHODS"."NAME" "method_NAME","TBL_IMPACT_CATEGORIES"."REF_ID" "REF_ID","TBL_IMPACT_CATEGORIES"."NAME" "category_NAME" FROM "TBL_IMPACT_CATEGORIES" LEFT JOIN "TBL_IMPACT_METHODS" ON "TBL_IMPACT_CATEGORIES"."F_IMPACT_METHOD"="TBL_IMPACT_METHODS"."ID"
SELECT "TBL_PROCESSES"."REF_ID" "P_m","TBL_UNITS"."NAME" "UNITS" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (SELECT "ID" FROM "TBL_PROCESSES" WHERE "REF_ID" IN ('64867712-23c4-3be5-a50e-3631e74571a6'))) "sq0" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="sq0".

,Flow,F_m,P_m,UNITS
0,1000.0,lemon,"lemon production | lemon | APOS, S | Spain",kg


We can find a mapping between openLCA reference units and `pyomo` units in the `utils` package of `mola`. 

In [27]:
unit_map = mb.map_reference_units()
{k: str(pu.get_units(v)) for k, v in unit_map.items()}

{'kg': 'kg',
 'm3': 'm ** 3',
 'MJ': 'MJ',
 'kBq': 'kBq',
 'm2': 'm',
 'm2*a': 'a * m ** 2',
 'm3*a': 'a * m ** 3',
 'Item(s)': 'count',
 'd': 'd',
 'kg*a': 'a * kg',
 't*km': 'km * t',
 'm': 'm',
 'p*km': 'count * km',
 'm*a': 'a * m'}

# Costs

Cost data in openLCA is in the `exchanges` table. This table is large so we only load the first 1000 rows here using an iterator and then show the table column names.

In [28]:
iter = dv.get_table(conn, 'TBL_EXCHANGES', chunk_size=1000)
exchanges_dfr = next(iter)
exchanges_dfr.columns

SELECT * FROM "TBL_EXCHANGES"


Index(['ID', 'F_OWNER', 'INTERNAL_ID', 'F_FLOW', 'F_UNIT', 'IS_INPUT',
       'F_FLOW_PROPERTY_FACTOR', 'RESULTING_AMOUNT_VALUE',
       'RESULTING_AMOUNT_FORMULA', 'AVOIDED_PRODUCT', 'F_DEFAULT_PROVIDER',
       'DESCRIPTION', 'COST_VALUE', 'COST_FORMULA', 'F_CURRENCY',
       'DISTRIBUTION_TYPE', 'PARAMETER1_VALUE', 'PARAMETER1_FORMULA',
       'PARAMETER2_VALUE', 'PARAMETER2_FORMULA', 'PARAMETER3_VALUE',
       'PARAMETER3_FORMULA', 'DQ_ENTRY', 'BASE_UNCERTAINTY'],
      dtype='object')

The columns `COST_VALUE` and `F_CURRENCY` give cost data. The currency column `F_CURRENCY` is a foreign key pointing to the currency table shown below.

In [29]:
currency_dfr = dv.get_table(conn, 'TBL_CURRENCIES')
currency_dfr

SELECT * FROM "TBL_CURRENCIES"


,ID,NAME,REF_ID,VERSION,LAST_CHANGE,F_CATEGORY,DESCRIPTION,CODE,CONVERSION_FACTOR,F_REFERENCE_CURRENCY
0,124988,Pound sterling,0b705d37-d71c-4c8f-8e02-2b36663635c6,None,None,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,GBP,1.528351,124989
1,124990,Swedish Krona,1ab0ca1b-79e7-4d5b-b501-cbf043ff302d,None,None,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,SEK,0.118590,124989
2,124991,Swiss Franc,24217f9a-0f63-4dc8-8440-c5829145f263,None,None,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,CHF,1.039069,124989
3,124992,South African Rand,2f1a6e69-442b-41bd-91dc-cb9e2e4e75c0,None,None,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,ZAR,0.078328,124989
4,124993,Hong Kong Dollar,30abeb04-b361-4ff3-8f63-b36796da7cb2,None,None,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,HKD,0.128992,124989
5,124994,Australian Dollar,3eee2da0-75ae-4f84-972c-e15e58167ef2,None,None,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,AUD,0.751145,124989
6,124989,US Dollar,505a07ff-16d7-4a83-b131-66998dad1732,None,None,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,USD,1.000000,124989
7,124995,Japanese Yen,7ddb41a4-823d-4432-bf6c-3ea8604b41be,None,None,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,JPY,0.008261,124989
8,124996,Euro,7de74e65-ff97-404f-bf57-5d2a9134fb43,None,None,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,EUR,1.109755,124989
9,124997,Singapore Dollar,a66b1ada-8042-44e9-88ac-6c89d2dc8e06,None,None,None,C:/data/CSV/system/juice_ecoinvent_36_apos_lci...,SGD,0.727484,124989


We demonstrate a function below to obtain the cost of a product flow from a given process reference id. First we obtain a process reference id.

In [30]:
oil_production_dfr = dv.get_processes(conn, name=['petroleum production, onshore%'], location=['Africa'])
oil_production_dfr

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE 'petroleum production, onshore%' AND "TBL_LOCATIONS"."NAME" LIKE 'Africa'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,29432025,cd177b7d-e908-3e69-b40c-4827b4abaa4d,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Africa,29432027


In [31]:
product_exchange_dfr = dv.get_exchanges(conn, id=29432027, columns=['ID', 'F_UNIT', 'F_OWNER', 'COST_VALUE', 'F_CURRENCY'])
product_exchange_dfr

SELECT "ID","F_UNIT","F_OWNER","COST_VALUE","F_CURRENCY" FROM "TBL_EXCHANGES" WHERE "ID"=29432027


,ID,F_UNIT,F_OWNER,COST_VALUE,F_CURRENCY
0,29432027,1056,29432025,0.192,124996.0


We join this to the unit and currency tables to get a price in currency per unit.

In [32]:
pe_dfr = product_exchange_dfr.merge(currency_dfr[['ID', 'NAME']], left_on='F_CURRENCY', right_on='ID')
pe_dfr = pe_dfr.merge(units_dfr[['ID', 'NAME']], left_on='F_UNIT', right_on='ID')[['ID_x', 'COST_VALUE', 'NAME_x', 'NAME_y']]
pe_dfr.columns = ['ID', 'COST', 'CURRENCY', 'UNIT']
pe_dfr

,ID,COST,CURRENCY,UNIT
0,29432027,0.192,Euro,kg


So the cost to extract 5 kg of crude oil in Africa (Libya and Algeria) is about 1 Euro according to the Ecoinvent database. The `mola` package contains a function to find this information more quickly given the process reference id.

In [33]:
from importlib import reload  
reload(dv)
ref_id = ['cd177b7d-e908-3e69-b40c-4827b4abaa4d']
dv.get_process_product_flow_costs(conn, process_ref_ids=ref_id)

SELECT "TBL_PROCESSES"."REF_ID" "PROCESS_REF_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","TBL_LOCATIONS"."NAME" "LOCATION","TBL_FLOWS"."REF_ID" "FLOW_REF_ID","TBL_FLOWS"."NAME" "FLOW_NAME","sq0"."COST_VALUE","TBL_CURRENCIES"."NAME" "CURRENCY","TBL_UNITS"."NAME" "UNITS" FROM (SELECT "F_OWNER","COST_VALUE","F_CURRENCY","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (SELECT "ID" FROM "TBL_PROCESSES" WHERE "REF_ID" IN ('cd177b7d-e908-3e69-b40c-4827b4abaa4d'))) "sq0" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="sq0"."F_FLOW" LEFT JOIN "TBL_PROCESSES" ON "TBL_PROCESSES"."ID"="sq0"."F_OWNER" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" LEFT JOIN "TBL_UNITS" ON "TBL_UNITS"."ID"="sq0"."F_UNIT" LEFT JOIN "TBL_CURRENCIES" ON "TBL_CURRENCIES"."ID"="sq0"."F_CURRENCY" WHERE "TBL_FLOWS"."FlOW_TYPE"='PRODUCT_FLOW'


,PROCESS_REF_ID,PROCESS_NAME,LOCATION,FLOW_REF_ID,FLOW_NAME,COST_VALUE,CURRENCY,UNITS
0,cd177b7d-e908-3e69-b40c-4827b4abaa4d,"petroleum production, onshore | petroleum | AP...",Africa,e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,petroleum,0.192,Euro,kg
